In [ ]:
"""
Purpose: To test out the changes needed to make to datajoint table 
for v4 run: 

1) changing how the axon and dendrite skeleton are saved
2) Doing the complete axon preprocessing, saving that off
and then passing that to the neuron proofreading function
3) How to save off the suggested edits in form that we 
can create suggested edits fo rneuroglancer
4) Make sure webbing and bouton information in table
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-04-26 22:50:22,695 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-26 22:50:22,698 - settings - Setting database.user to celiib
INFO - 2021-04-26 22:50:22,699 - settings - Setting database.password to newceliipass
INFO - 2021-04-26 22:50:22,712 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-26 22:50:22,713 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-26 22:50:22,727 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-26 22:50:23,180 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-26 22:50:23,184 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-26 22:50:23,185 - settings - Setting database.user to celiib
INFO - 2021-04-26 22:50:23,186 - settings - Setting database.password to newceliipass
INFO - 2021-04-26 22:50:23,187 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-26 22:50:23,188 - settings - Setting database.user to celiib
INFO - 2021-04-26 22:50:23,188 - settings - Setting database.password to newceliipass
INFO - 2021-04-26 22:50:23,191 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-26 22:50:23,635 - settings - Setting enable_python_native_blobs to True


In [133]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

In [13]:
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su
import axon_utils as au

# Need a table to store the fully axon processed neurons

In [5]:
minnie.AutoProofreadVersion.describe()

proof_version        : tinyint unsigned             # key by which to lookup the decomposition process version
---
description          : varchar(256)                 # new parts of the iteration of the decomposition process



'proof_version        : tinyint unsigned             # key by which to lookup the decomposition process version\n---\ndescription          : varchar(256)                 # new parts of the iteration of the decomposition process\n'

# Testing Populating the Axon Decomposition Table

In [19]:
verbose = True

In [78]:
curr_table = (minnie.Decomposition() & "n_limbs = 10" & "n_somas=2")
curr_table

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134884772346,0,0.25,30.00,0,0,1,=BLOB=,385614,805879,0,1,1,1,2,10,106,51,3416069.9194424567,1568051.5996385978,22918.888,186.605,186.605,506.188,456.498,1063,0.0003111763005639809,10.028301886792454,2126475.354,56,0.0004998882296004264,18.982,207947661950.84503,109383813.78330484,60873.36233000306,97789.82933410713,3713351106.26509,1702.3648
864691134917423882,0,0.25,30.00,5,0,1,=BLOB=,654706,1323616,0,1,1,1,2,10,185,96,5033741.307436872,2332008.177201809,16644.122,225.065,222.719,440.921,420.612,1887,0.0003748702773446338,10.2,3783813.289,125,0.0004987032540653462,15.096,393234081987.2192,140608340.16618347,78119.6446083261,103925.33984972196,3145872655.8977532,6760.2047
864691134939984483,0,0.25,30.00,0,0,1,=BLOB=,850056,1712989,0,1,1,1,2,10,352,186,9694467.839110663,4280376.13648248,17357.902,67.644,67.644,405.872,330.68,2543,0.00026231455322804864,7.224431818181818,4105788.211,79,0.0006193695021060598,32.19,359654355154.724,73883426.51840337,37098.92705030805,87596.90872294825,4552586774.110431,5115.6062
864691134947441916,0,0.25,30.00,0,0,1,=BLOB=,810849,1642418,0,1,0,1,2,10,152,65,6424737.076339348,2598178.4804172376,34119.672,378.327,311.13,692.705,646.108,3770,0.0005867944408003775,24.80263157894737,5973018.884,132,0.0006311716191118609,28.561,650289383149.8154,99074108.83834966,101216.49733257781,108871.14134056293,4926434720.831935,4219.6827
864691134988401018,0,0.25,30.00,0,0,1,=BLOB=,1070648,2153386,0,2,1,1,2,10,264,123,12010311.912833858,3848738.203157431,25031.814,165.305,162.207,494.164,449.113,3147,0.000262024835228235,11.920454545454545,7919810.365,151,0.0003973579991141619,20.841,392357752255.1989,70381231.51502481,32668.40654121041,49541.30644202601,2598395710.2993307,3971.4798
864691134988413562,0,0.25,30.00,0,0,1,=BLOB=,804636,1650151,0,2,1,1,2,10,240,162,6822269.093434698,3099455.8510754304,13385.288,269.585,253.752,511.0

In [79]:
key = (curr_table & dict(segment_id = 864691135012573174)).proj().fetch1()
key

{'segment_id': 864691135012573174,
 'decimation_version': 0,
 'decimation_ratio': Decimal('0.25'),
 'ver': Decimal('30.00'),
 'process_version': 5,
 'index': 0}

In [80]:
verbose = True
# 1) Pull Down All of the Neurons
segment_id = key["segment_id"]

whole_pass_time = time.time()

#1) Pull Down all the Neuron Objects associated with a segment_id

neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

if verbose:
    print(f"Number of Neurons found ={len(neuron_objs)}")
    
#For each neuron:
#for split_index,neuron_obj_pre_split in zip(neuron_split_idxs,neuron_objs):


Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 2
Number of Neurons found =2


In [83]:
split_index = 1
neuron_obj = neuron_objs[split_index]

#Run the Axon Decomposition
neuron_obj_with_web = au.complete_axon_processing(neuron_obj,
                                     verbose=True)


Part 1: Axon like branchese 
{'L0': array([ 1,  4,  7,  8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]), 'L4': array([0, 1, 2]), 'L5': array([ 2, 23, 31, 40, 41, 42, 43, 44, 45, 46, 47])}
Limb L0 soma angle: 85.8 
Limb L1 soma angle: 92.04 
Limb L2 soma angle: 111.81 
Limb L3 soma angle: 130.51 
Limb L4 soma angle: 126.37 
Limb L5 soma angle: 36.0 

Part 2: possible_axon_limbs = [0, 1, 2, 3, 4]

Phase 2: Working on Limb L0
nodes_to_eliminate = [5]

Phase 2: Working on Limb L1
nodes_to_eliminate = [4]

Phase 2: Working on Limb L2
nodes_to_eliminate = [0]

Phase 2: Working on Limb L3
nodes_to_eliminate = []

Phase 2: Working on Limb L4
nodes_to_eliminate = []

----Working on Limb L0-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L1-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No endpoints were eliminated so don't need to a

100%|██████████| 1/1 [00:00<00:00,  8.19it/s]

root_curr = [1101932.   905051.1  941031.9]
Time for preparing soma vertices and root: 0.0007224082946777344
invalidation_d = 1500
smooth_neighborhood = 0
cc_vertex_thresh = 10


meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 0.391834020614624
branches_touching_root = [8]
combining close nodes
combine_threshold = 1300
length of Graph = 2916
After combining close endpoints max(kept_branches_idx) = 16, len(kept_branches_idx) = 16
empty_indices % = 0.0
 conflict_indices % = 0.018318373328448434
AFTER face_lookup_resolved_test
Decomposing first pass: 1.7792327404022217
Only Using MeshParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 4.00543212890625e-05
Total time for MAP sublimb processing 9.059906005859375e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 0.00012922286987304688
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.0019714832305908203
sbv[0].reshape(-1,3) = [[1101932.   905051.1  941031.9]]
closest_sk_pt_coord BEFORE = [1101265.01652063  904696

In [163]:
neuron_obj_with_web = n_obj

In [164]:
# Just trying to filter neuron to se what it would look like
filter_list = pru.v4_exc_filters()
inh_exc_class="excitatory"
perform_axon_classification = False
plot_limb_branch_filter_with_disconnect_effect = True

filtered_neuron,filtering_info = pru.proofread_neuron_class_predetermined(neuron_obj_with_web,
        inh_exc_class=inh_exc_class,
        perform_axon_classification=False,

        plot_limb_branch_filter_with_disconnect_effect = plot_limb_branch_filter_with_disconnect_effect,


        plot_final_filtered_neuron = False,

        #arguments for the axon high fidelity:
        plot_new_axon_limb_correspondence = False,
        plot_new_limb_object = False,
        plot_final_revised_axon_branch = False,

        verbose = False,
        verbose_outline = verbose,
        filter_list = filter_list,
        high_fidelity_axon_on_excitatory = perform_axon_classification,
                                                                             )


--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': True}



--- Working on filter 1:
function = high_degree_branching
function __name__ = filter_away_high_degree_branching
function arguments = {'offset': 1500, 'comparison_distance': 2000, 'worst_case_match_threshold': 65, 'width_diff_max': None, 'width_diff_perc': 60, 'match_threshold': 45, 'angle_buffer': 15, 'max_degree_to_resolve': 6, 'match_method': 'best_match'}




 ----- Working on coordinate [491775.73454331 796768.97696681 963426.61025879]--------
coordinate [491775.73454331 796768.97696681 963426.61025879] had error branches [7 9]--------


 ----- Working on coordinate [517441.17096419 784951.98283524 881829.82393316]--------
coordinate [517441.17096419 784951.98283524 881829.82393316] had error branches [21 23 24]--------


 ----- Working on coordinate [525906.87270703 674386.73804382 840716.1500395 ]------


--- Working on filter 2:
function = axon_webbing_t_merges
function __name__ = filter_away_webbing_t_merges
function arguments = {'child_width_maximum': 75, 'parent_width_maximum': 75, 'axon_only': True, 'error_if_web_is_none': True, 'web_size_threshold': 120, 'web_size_type': 'ray_trace_median', 'web_above_threshold': True}


limb_branch_dict_to_cancel = {}

--- Working on filter 3:
function = thick_t_merge
function __name__ = filter_away_thick_t_merge
function arguments = {}


limb_branch_dict_to_cancel = {}

--- Working on filter 4:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': True, 'skip_double_back_errors_for_axon': False, 'width_jump_threshold': 250, 'running_width_jump_method': True, 'double_back_axon_like_threshold': 145, 'double_back_threshold': 120, 'allow_axon_double_back_angle_with_top': None, 'allow_axon_double_back_angle_with_top_width_min': 140, 'skeletal_len


--- Working on filter 5:
function = axon_fork_divergence
function __name__ = filter_away_small_axon_fork_divergence
function arguments = {'divergence_threshold_mean': 165}


limb_branch_dict_to_cancel = {}



 ---- Total time for applying filter: 59.70293164253235 -----


In [86]:
nviz.plot_original_vs_proofread(original = neuron_obj,
    proofread = filtered_neuron,
    original_color = "red",
    proofread_color = "blue",
    mesh_alpha = 0.3,
    plot_mesh= True,
    plot_skeleton = True)

In [89]:
filtered_neuron.description

'0_25_soma_1_split_axon_v1'

In [127]:
minnie.Decomposition() & "n_somas = 2"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134765143804,0,0.25,30.00,0,0,1,=BLOB=,44570,90882,0,1,1,1,2,1,1,1,28585.105732166514,28585.105732166514,28585.106,1726.047,1726.047,1726.047,1726.047,4,0.00013993301397863443,4.0,28585.106,1,0.00013993301266750595,4.0,174766472.25666714,13946453.78167041,6113.899801322207,6113.899744036882,174766472.25666714,89.0604
864691134768966140,0,0.25,30.00,5,0,1,=BLOB=,81637,164711,0,1,1,1,2,2,5,4,61206.675450776704,40401.85629631666,4713.508,108.165,108.165,1174.845,1135.231,2,3.2676174375921285e-05,0.4,6413.497,2,0.0003118423537112436,1.0,806371663.6001841,403185831.80009204,13174.570545800676,125730.41877156628,403185831.80009204,58.3114
864691134831878402,0,0.25,30.00,0,0,1,=BLOB=,141431,288006,0,1,0,1,2,3,5,3,167837.6994974118,105050.86635215634,42893.926,114.293,114.293,1035.029,1035.029,0,0.0,0.0,19892.907,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.6019
864691134874936141,0,0.25,30.00,0,0,1,=BLOB=,64451,129722,0,2,2,1,2,3,7,3,69751.43119438276,56515.80323426945,5610.567,124.571,124.571,1568.841,1568.841,4,5.7346493563018516e-05,0.5714285714285714,41954.196,3,9.534207257839e-05,1.333,123117238.4626452,18270225.275904972,1765.0854807486746,2934.5631712891172,41039079.4875484,143.5662
864691134884772346,0,0.25,30.00,0,0,1,=BLOB=,385614,805879,0,1,1,1,2,10,106,51,3416069.9194424567,1568051.5996385978,22918.888,186.605,186.605,506.188,456.498,1063,0.0003111763005639809,10.028301886792454,2126475.354,56,0.0004998882296004264,18.982,207947661950.84503,109383813.78330484,60873.36233000306,97789.82933410713,3713351106.26509,1702.3648
864691134884780538,0,0.25,30.00,0,0,1,=BLOB=,368083,762245,0,3,3,1,2,9,113,58,2596256.398943132,1236697.6953156113,18046.919,349.046,300.773,604.522,586.524,1347,0.0005188239499566871,11.920353982300885,2333114.813,95,0.0005773397830636464,14.179,226915259840.80365,94399194.066775,87400.94388719655,97258.50548650374,2388581682.53477

In [168]:
minnie.DecompositionAxon()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,axon_version key by which to lookup the finer axon processing method,decomposition saved neuron object with high fidelity axon,axon_length length (in um) of the classified axon skeleton,run_time the amount of time to run (seconds)
864691134384033872,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.26
864691134409056935,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.38
864691134428609977,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.13
864691134488295985,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.31
864691134684569005,0,0.25,30.00,5,0,0,4,=BLOB=,0.0,0.15
864691134743989494,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.61
864691134751332440,0,0.25,30.00,5,0,0,4,=BLOB=,0.0,0.46
864691134762254869,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.23
864691134792072188,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.15
864691134800869987,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.25


In [154]:
minnie.DecompositionAxon() & "axon_length>1000" & "segment_id=864691134917425418"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,axon_version key by which to lookup the finer axon processing method,decomposition saved neuron object with high fidelity axon,axon_length length (in um) of the classified axon skeleton,run_time the amount of time to run (seconds)
864691134917425418,0,0.25,30.00,5,0,0,4,=BLOB=,1279.4147732058873,181.57


In [155]:
n_obj = du.decomposition_with_spine_recalculation(864691134917425418,0)

**Using table __decomposition_axon for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 1


In [157]:
orig_seg_mesh = du.fetch_segment_id_mesh(n_obj.segment_id)

In [159]:
orig_seg_mesh

<trimesh.Trimesh(vertices.shape=(203816, 3), faces.shape=(421561, 3))>

In [165]:
nviz.plot_original_vs_proofread(original = n_obj,
    proofread = filtered_neuron,
    original_color = "red",
    proofread_color = "blue",
    mesh_alpha = 0.3,
    plot_mesh= True,
    plot_skeleton = False)

original = <trimesh.Trimesh(vertices.shape=(203816, 3), faces.shape=(421561, 3))>


In [94]:
x = None
try:
    du.decomposition_with_spine_recalculation(864691136231316382,0)
except Exception as e:
    x = e.args[0]
    print(e)

**Using table __decomposition for table_to_neuron_objs**
'/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691136231316382_0_25.pbz2' downloaded but did not pass checksum'


In [119]:
import datajoint_utils as du
du.datajoint_filepath_error_to_filepath(error = x)

'/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691136231316382_0_25.pbz2'

In [59]:
save_time = time.time()
ret_file_path = filtered_neuron.save_compressed_neuron(
                                #output_folder=str(du.get_decomposition_path()),
                                output_folder = "./",
                                #file_name=f"{neuron_obj_with_web.segment_id}_validation_full_axon",
                                  return_file_path=True,
                                 export_mesh=False,
                                 suppress_output=False)

ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
print(f"ret_file_path_str = {ret_file_path_str}")
print(f"Save time = {time.time() - save_time}")

Total time for soma meshes compression = 0.5879535675048828
Total time for insignificant_limbs,inside_pieces,non_soma_touching_meshes,not_processed_soma_containing_meshes compression = 2.09808349609375e-05
Total time for limb_meshes compression = 1.5303874015808105
    Starting Limb Correspondence Compression
Total time for new_limb_correspondence compression = 9.438011646270752
Total time for compression = 39.20625686645508
Saved object at /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/v4/864691135081713143_0_25_soma_0_split_axon_v1.pbz2
File size is 1.96837 MB


---Finished outputing neuron at location: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/v4/864691135081713143_0_25_soma_0_split_axon_v1---
Saved File at location: 864691135081713143_0_25_soma_0_split_axon_v1
ret_file_path_str = /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/v4/864691135081713143_0_25_soma_0_split_axon_v1.pbz2
Save time = 40.20885133743286


In [62]:
n_dict = dict(key,
              split_index = split_index,
              axon_version = au.axon_version,
             decomposition=ret_file_path_str,
             axon_length=filtered_neuron.axon_length,
              run_time = np.round(time.time() - whole_pass_time,2)
             )

n_dict

{'segment_id': 864691135081713143,
 'decimation_version': 0,
 'decimation_ratio': Decimal('0.25'),
 'ver': Decimal('30.00'),
 'process_version': 5,
 'index': 0,
 'split_index': 0,
 'axon_version': 1,
 'decomposition': '/neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/v4/864691135081713143_0_25_soma_0_split_axon_v1.pbz2',
 'axon_length': 397.14731961081964}

# Fetching an example neuron

In [69]:
du.decomposition_with_spine_recalculation?

In [68]:
table_to_neuron_objs(table,
    return_one=False,
    return_split_index=False)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134765143804,0,0.25,30.00,0,0,1,=BLOB=,44570,90882,0,1,1,1,2,1,1,1,28585.105732166514,28585.105732166514,28585.106,1726.047,1726.047,1726.047,1726.047,4,0.00013993301397863443,4.0,28585.106,1,0.00013993301266750595,4.0,174766472.25666714,13946453.78167041,6113.899801322207,6113.899744036882,174766472.25666714,89.0604
864691134768966140,0,0.25,30.00,5,0,1,=BLOB=,81637,164711,0,1,1,1,2,2,5,4,61206.675450776704,40401.85629631666,4713.508,108.165,108.165,1174.845,1135.231,2,3.2676174375921285e-05,0.4,6413.497,2,0.0003118423537112436,1.0,806371663.6001841,403185831.80009204,13174.570545800676,125730.41877156628,403185831.80009204,58.3114
864691134831878402,0,0.25,30.00,0,0,1,=BLOB=,141431,288006,0,1,0,1,2,3,5,3,167837.6994974118,105050.86635215634,42893.926,114.293,114.293,1035.029,1035.029,0,0.0,0.0,19892.907,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.6019
864691134874936141,0,0.25,30.00,0,0,1,=BLOB=,64451,129722,0,2,2,1,2,3,7,3,69751.43119438276,56515.80323426945,5610.567,124.571,124.571,1568.841,1568.841,4,5.7346493563018516e-05,0.5714285714285714,41954.196,3,9.534207257839e-05,1.333,123117238.4626452,18270225.275904972,1765.0854807486746,2934.5631712891172,41039079.4875484,143.5662
864691134884772346,0,0.25,30.00,0,0,1,=BLOB=,385614,805879,0,1,1,1,2,10,106,51,3416069.9194424567,1568051.5996385978,22918.888,186.605,186.605,506.188,456.498,1063,0.0003111763005639809,10.028301886792454,2126475.354,56,0.0004998882296004264,18.982,207947661950.84503,109383813.78330484,60873.36233000306,97789.82933410713,3713351106.26509,1702.3648
864691134884780538,0,0.25,30.00,0,0,1,=BLOB=,368083,762245,0,3,3,1,2,9,113,58,2596256.398943132,1236697.6953156113,18046.919,349.046,300.773,604.522,586.524,1347,0.0005188239499566871,11.920353982300885,2333114.813,95,0.0005773397830636464,14.179,226915259840.80365,94399194.066775,87400.94388719655,97258.50548650374,2388581682.53477

In [70]:
du.table_to_neuron_objs

<function datajoint_utils.table_to_neuron_objs(table, return_one=False, return_split_index=False)>

In [72]:
curr_table = minnie.DecompositionSplit() & "segment_id = 864691134917398026"
curr_table

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,split_version the version of the splitting algorithm used,multiplicity the number of somas found for this base segment,n_splits the number of cuts required to help split the neuron,split_success the successfulness of the splitting,n_error_limbs_cancelled number of limbs that couldn't be resolved and cancelled out,n_same_soma_limbs_cancelled number of same soma touching limbs that couldn't be resolved and cancelled out,n_multi_soma_limbs_cancelled number of multi soma touching limbs that couldn't be resolved and cancelled out,error_imbs_cancelled_area the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas,error_imbs_cancelled_skeletal_length the total skeletal length (in microns) of the limbs that were called out because could not be resolved,split_results will store the results of how to split the limbs of neuron objects from original neuron,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134917398026,0,0.25,30.00,0,0,0,1,2,4,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,306127,610325,0,0,0,0,1,30727,581,8,66,24,2903250.0637134355,988989.093265544,27202.235,284.14,250.233,428.627,407.311,1400,0.0004822181931546447,21.21212121212121,2269968.166,48,0.0006167487372596044,29.167,305731039439.93286,126978331.48126602,105306.47816429703,134685.16608260348,6369396654.998601,2102.3057
864691134917398026,0,0.25,30.00,0,0,1,1,2,4,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,364589,727008,0,0,0,0,1,40244,656,12,133,64,4527732.153995185,2540786.0100990958,18574.088,129.661,129.661,328.214,316.796,657,0.00014510575662482058,4.93984962406015,2194997.257,58,0.0002993170027455756,11.328,57952932580.05713,49163520.60981496,12799.549666143692,26402.280182921033,999188492.7596056,3151.1663


AttributeError: module 'minnie' has no attribute 'DecompositionAxon'

In [ ]:
"""
Need a function that will go and get the 


"""
